<a href="https://colab.research.google.com/github/Filbra/Colab_Earth_Engine/blob/main/Umm_al_Quwain_geepy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Mounting your Google Drive in the runtime's virtual machine

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Install Python Modules
! pip install geemap


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=7b99fa7c64fd2bf3cd55e44149acb7468c57dbf8660f05d23b184

In [ ]:
## Import Python Modules

import ee
import geemap
import os

In [ ]:
## Open a folium map object centered on Umm al Quwain (UAE) with 'geemap'

Map = geemap.Map()

Umm_al_Quwain = ee.Geometry.Polygon(
[[55.495377,25.510889],                                      
[55.495377,25.670665],                                      
[55.730896,25.670665],                                      
[55.730896,25.510889]], 'EPSG:4326', False)

Map.centerObject(Umm_al_Quwain, 12)
Map                                 

Map(center=[25.590774934310463, 55.61313649999971], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
## To customise the AOI, draw a polygon in the map above and run this cell.
CustomDraw = ee.Feature(Map.draw_last_feature)
Geom = CustomDraw.geometry()
Coords = Geom.coordinates()
ROI = ee.Geometry.Polygon(Coords)
#print(ROI)

**Multi - temporal approach, Spectral indices & Spectral decomposition analysis** (*F. Brandolini*, filippo.brandolini@newcastle.ac.uk)

In [ ]:
## This is the cloud masking function provided by GEE but adapted for use in Python.

def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  
  return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image,["system:time_start"])

In [ ]:
## Sentinel 2

## 2022

S2_coll = (ee.ImageCollection('COPERNICUS/S2')
          .filterDate('2022-01-01', '2022-12-31')
          .filterBounds(ROI)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
          .select(['B2','B3','B4','B8','B11','B12','QA60'])
          .map(maskS2clouds))



In [ ]:
## Reducing the images collection in a single 6-bands image

S2_6bands = (S2_coll.select(['B2','B3','B4','B8','B11','B12']).median().clip(ROI))

In [ ]:
## RGB composite

## bands 4–3–2 (Red/Green/Blue), visualisation parameters:

RGB_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': [0.1787, 0.1989, 0.23],
    'max': [0.453798, 0.387999, 0.3572],
  }

In [ ]:
##  NDVI = (NIR - RED) / (NIR + RED)
## NDVI (Sentinel 2) = (B8 - B4)/ (B8 + B4)

NDVI = S2_6bands.expression('(nir - red)/(nir + red)' ,{
    'nir':S2_6bands.select('B8'),
    'red':S2_6bands.select('B4'),
}).rename('NDVI')

## NDVI visualisation parameters:

NDVI_vis = {
    'min': -0.2938584, 
    'max': -0.037054, 
    'palette': ['#d53e4f','#f46d43','#fdae61','#fee08b','#ffffbf','#e6f598','#abdda4','#66c2a5','#3288bd']} #Spectral


In [ ]:
## Land Surface Water Index (LSWI) (NIR - SWIR)/(NIR + SWIR)
## LSWI (Sentinel 2) = (B8-B12)/(B8+B12) 

LSWI = S2_6bands.expression('(nir - swir)/(nir + swir)' ,{
    'nir':S2_6bands.select('B8'),
    'swir':S2_6bands.select('B12'),
}).rename('LSWI')

## LSWI visualisation parameters:

LSWI_vis = {
    'min': -0.2938584, 
    'max': -0.037054, 
    'palette': ['#d53e4f','#f46d43','#fdae61','#fee08b','#ffffbf','#e6f598','#abdda4','#66c2a5','#3288bd']} #Spectral

In [ ]:
## Modified Normalized Difference Water Index (MNDWI) (GREEN - SWIR) / (GREEN+SWIR)
## MNDWI (Sentinel 2) = (B3-B12)/(B3+B12) 

MNDWI = S2_6bands.expression('(green - swir)/(green + swir)' ,{
    'green':S2_6bands.select('B3'),
    'swir':S2_6bands.select('B12'),
}).rename('MNDWI')

## MNDWI visualisation parameters:

MNDWI_vis = {
    'min': -0.2938584, 
    'max': -0.037054, 
    'palette': ['#d53e4f','#f46d43','#fdae61','#fee08b','#ffffbf','#e6f598','#abdda4','#66c2a5','#3288bd']} #Spectral

In [ ]:
## Hue, Saturation and Value (HSV)
## Convert the RGB bands to the HSV color space.

HSV = (S2_coll.select(['B2','B3','B4']).mean().clip(ROI)).rgbToHsv()
HSV_vis = {'bands': ['hue', 'saturation', 'value'],
    'min': [0.00443054, 0.137734, 0.125822], 
    'max': [0.993581, 0.310943, 0.147081]
}


In [ ]:
## Tassellation Cap Transformation (6 Bands)

## TCT components were computed using both the six bands described in the formulas 

## Create an Array of Tasseled Cap coefficients.
## • TCT-band 1 (brightness, measure of soil)
## • TCT-band 2 (greenness, measure of vegetation)
## • TCT-band 3 (wetness, interrelationship of soil and canopy moisture)

coefficients_6 = ee.Array([
  [0.3510, 0.3813, 0.3437, 0.7196, 0.2396, 0.1949], #TCTb
  [-0.3599, -0.3533, -0.4734, 0.6633, - 0.0087, -0.2856], #TCTg
  [0.2578, 0.2305, 0.0883, 0.1071, -0.7611, -0.5308], #TCTw
]);

## Make an Array Image, with a 1-D Array per pixel.
arrayImage1D = S2_6bands.toArray();

## Make an Array Image with a 2-D Array per pixel, 6x1.
arrayImage2D = arrayImage1D.toArray(1);

## Do a matrix multiplication: 6x6 times 6x1.
TCT = (ee.Image(coefficients_6)
          .matrixMultiply(arrayImage2D)
          .arrayProject([0])
          .arrayFlatten([['TCTb', 'TCTg', 'TCTw']]))

## Display the first three bands of the result and the input imagery.

TCT_vis = {
    'bands': ['TCTb', 'TCTg', 'TCTw'],
    'min': [0.255928, -0.0965299, -0.126619], 
    'max': [0.356473, -0.0218605, -0.0655193]
}

In [ ]:
# PCA Principal Component Analysis

scale = 10
bandNames = S2_6bands.bandNames()

meanDict = S2_6bands.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': ROI,
    'scale': scale,
    'maxPixels': 1e9
    })

means = ee.Image.constant(meanDict.values(bandNames))
centered = S2_6bands.subtract(means)

def getNewBandNames(prefix):
  seq = ee.List.sequence(1, bandNames.length())

  def func_zwm(b):
    return ee.String(prefix).cat(ee.Number(b).int().format())
  
  return seq.map(func_zwm)

def getPrincipalComponents(centered, scale, geometry):

## Collapse the bands of the image into a 1D array per pixel.
  arrays = centered.toArray()

##Compute the covariance of the bands within the region.
  covar = arrays.reduceRegion(**{
      'reducer': ee.Reducer.centeredCovariance(),
      'geometry': ROI,
      'scale': scale,
      'maxPixels': 1e9
      })
  
##Get the 'array' covariance result and cast to an array.
##This represents the band-to-band covariance within the region.

  covarArray = ee.Array(covar.get('array'))

## Perform an eigen analysis and slice apart the values and vectors.

  eigens = covarArray.eigen()

## This is a P-length vector of Eigenvalues.

  eigenValues = eigens.slice(1, 0, 1)

## This is a PxP matrix with eigenvectors in rows.

  eigenVectors = eigens.slice(1, 1)

## Convert the array image to 2D arrays for matrix computations.

  arrayImage = arrays.toArray(1)

## Left multiply the image array by the matrix of eigenvectors.

  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

## Turn the square roots of the Eigenvalues into a P-band image.

  sdImage = ee.Image(eigenValues.sqrt()) \
  .arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

## Turn the PCs into a P-band image, normalized by SD.
  return principalComponents \
  .arrayProject([0]) \
  .arrayFlatten([getNewBandNames('pc')]) \
  .divide(sdImage)

# Get the PCs at the specified scale and in the specified region
PCA = getPrincipalComponents(centered, scale, ROI)

# Vis Parameters PCA

PCA_vis = {
    'bands': ['pc1', 'pc2', 'pc3'],
    'min': [-2.02982, -1.50828, -1.16942], 
    'max': [1.63844, 2.30986, 1.32921]
}

In [ ]:
#Add Layers
Map.add_basemap(basemap='SATELLITE')
Map.addLayer(S2_6bands, RGB_vis, "RGB", True)
Map.addLayer(NDVI, NDVI_vis, "NDVI", True)
Map.addLayer(LSWI, LSWI_vis, "LSWI", True)
Map.addLayer(MNDWI, MNDWI_vis, "MNDWI", True)
Map.addLayer(HSV, HSV_vis, "HSV")
Map.addLayer(TCT, TCT_vis, "TCT", True)
Map.addLayer(PCA, PCA_vis, "PCA", True)

In [ ]:
# Set map center
Map.center_object(S2_6bands, zoom = 16) 

Map.add_layer_control()
Map

Map(bottom=447440.0, center=[25.54414464917812, 55.58361049999194], controls=(WidgetControl(options=['position…

In [ ]:
## Export outputs in a Drive Folder

#Set Coordinates Reference Systems (CRS), For example: Italy --> 'EPSG:32632';  WGS 84 --> 'EPSG:4326'. UAE--> 'EPSG:32640'

image_list = [S2_6bands, NDVI, LSWI, MNDWI, HSV, TCT, PCA]
prefix_list = ['RGB', 'NDVI', 'LSWI', 'MNDWI', 'HSV', 'TCT', 'PCA']

pjcrs = 'EPSG:32640'
res = 10
maxpix = 1e13

for image, prefix in zip(image_list, prefix_list):
    geemap.ee_export_image_to_drive(
        image=image,
        description='export',
        folder='GEE_outputs',
        fileNamePrefix=prefix,
        crs=pjcrs,
        region=ROI,
        scale=res,
        maxPixels=maxpix
    )

In [ ]:
# ## Export outputs in a Drive Folder

# #Set Coordinates Reference Systems (CRS), For example: Italy --> 'EPSG:32632';  WGS 84 --> 'EPSG:4326'. UAE--> 'EPSG:32640'

# pjcrs = 'EPSG:32640'
# res = 10
# maxpix = 1e13

# ##RGB

# geemap.ee_export_image_to_drive(
#     image= S2_6bands,
#     description='export',
#     folder='GEE_outputs',
#     fileNamePrefix='RGB',
#     crs= pjcrs,
#     region= ROI,
#     scale= res,
#     maxPixels=maxpix
# )

# ##NDVI

# geemap.ee_export_image_to_drive(
#     image= NDVI,
#     description='export',
#     folder='GEE_outputs',
#     fileNamePrefix='NDVI',
#     crs= pjcrs,
#     region= ROI,
#     scale= res,
#     maxPixels=maxpix
# )

# ##LSWI

# geemap.ee_export_image_to_drive(
#     image= LSWI,
#     description='export',
#     folder='GEE_outputs',
#     fileNamePrefix='LSWI',
#     crs= pjcrs,
#     region= ROI,
#     scale= res,
#     maxPixels=maxpix
# )

# ##MNDWI

# geemap.ee_export_image_to_drive(
#     image= MNDWI,
#     description='export',
#     folder='GEE_outputs',
#     fileNamePrefix='MNDWI',
#     crs= pjcrs,
#     region= ROI,
#     scale= res,
#     maxPixels=maxpix
# )

# ## HSV

# geemap.ee_export_image_to_drive(
#     image= HSV,
#     description='export',
#     folder='GEE_outputs',
#     fileNamePrefix='HSV',
#     crs= pjcrs,
#     region= ROI,
#     scale= res,
#     maxPixels=maxpix
# )

# ## TCT

# geemap.ee_export_image_to_drive(
#     image= TCT,
#     description='export',
#     folder='GEE_outputs',
#     fileNamePrefix='TCT',
#     crs= pjcrs,
#     region= ROI,
#     scale= res,
#     maxPixels=maxpix
# )

# ##PCA

# geemap.ee_export_image_to_drive(
#     image= PCA,
#     description='export',
#     folder='GEE_outputs',
#     fileNamePrefix='PCA',
#     crs= pjcrs,
#     region= ROI,
#     scale= res,
#     maxPixels=maxpix
# )

In [ ]:
## Generate image download URL. If your ara running the code in Colab, the outputs are saved automatically in the sample_data folder in your Drive
## !!! It works only with small areas / low resolution !!!

#geemap.ee_export_image(ee_object= S2_6bands, filename='RGB.tif',scale= res,crs= pjcrs,region= ROI, file_per_band= False)
#geemap.ee_export_image(ee_object= NDVI, filename='NDVI.tif',scale= res,crs= pjcrs,region= ROI, file_per_band= False)
#geemap.ee_export_image(ee_object= LSWI, filename='LSWI.tif',scale= res,crs= pjcrs,region= ROI, file_per_band= False)
#geemap.ee_export_image(ee_object= MNDWI, filename='MNDWI.tif',scale= res,crs= pjcrs,region= ROI, file_per_band= False)
#geemap.ee_export_image(ee_object= HSV, filename='HSV.tif',scale= res,crs= pjcrs,region= ROI, file_per_band= False)
#geemap.ee_export_image(ee_object= TCT, filename='TCT.tif',scale= res,crs= pjcrs,region= ROI, file_per_band= False)
#geemap.ee_export_image(ee_object= PCA, filename='PCA.tif',scale= res,crs= pjcrs,region= ROI, file_per_band= False)